# Text classification with Generative Models
Text
 classification is a common task in NLP, it involves categorizing text 
into predefined categories or classes based on its content. This task is
 essential in various applications, such as sentiment analysis, spam 
filtering, topic classification...

![](Text%20classification%20with%20Generative%20Models%20-%20Computer%20Science%20Adventures_files/text-classification-huh.jpg)

Now with all the generative models it's tempting to use them for 
classification tasks, but are they good at it? How can we measure the 
success of a classification model ? Let's find out 🤔

For this example we will use the `rotten_tomatoes` dataset, it contains 50000 movie reviews with their corresponding sentiment (positive or negative).

In [ ]:
from datasets import load_dataset

# Load the Rotten Tomatoes sentiment dataset
data = load_dataset("rotten_tomatoes")
data

/Users/william/python-envs/data-science/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 1066/1066 [00:00<00:00, 516177.33 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

## Using a task-specific model
Using
 specific task models is the easiest way to solve our problem, we just 
need to find a model that fits our needs, download it and use it in a 
pipeline to test it on our data.

> 
For this example we will use a `roberta` model to classify our data.

We will use a `pipeline` object if you are not familiar with this read the [official doc](https://huggingface.co/docs/transformers/v4.24.0/main_classes/pipelines)

In [ ]:
from transformers import pipeline
import torch

model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"


def select_torch_device() -> torch.device:
    """Pick the best available device (CUDA > MPS > CPU)."""
    if torch.cuda.is_available():
        return torch.device("cuda")
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")


device = select_torch_device()
print(f"Using device: {device}")

# Load model into a pipeline
# NOTE: `return_all_scores=True` is deprecated upstream but we keep it here
# to preserve the exact output structure used later in the notebook.
pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device=device,
)

Using device: cpu


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
/Users/william/python-envs/data-science/lib/python3.14/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `t

Now let's run an inference loop to get the predictions for our dataset

In [ ]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

# Run inference on the test split.
# This RoBERTa checkpoint outputs 3 labels (negative / neutral / positive).
# We reproduce the original logic by only comparing negative vs positive scores.
y_pred = []

test_texts = KeyDataset(data["test"], "text")
for scores in tqdm(pipe(test_texts), total=len(data["test"])):
    negative_score = scores[0]["score"]
    positive_score = scores[2]["score"]

    # 0 -> negative, 1 -> positive
    y_pred.append(int(np.argmax([negative_score, positive_score])))

100%|██████████| 1066/1066 [00:29<00:00, 36.01it/s]


### Evaluation
Then
 we will define a function to evaluate how well the model performed by 
comparing predictions to actual labels. For this we will use the `classification_report` from sklearn

In [ ]:
from sklearn.metrics import classification_report


def evaluate_performance(y_true, y_pred) -> str:
    """Print (and return) a classification report for Rotten Tomatoes sentiment."""
    report = classification_report(
        y_true,
        y_pred,
        target_names=["Negative Review", "Positive Review"],
    )
    print(report)
    return report

In [6]:
evaluate_performance(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.76      0.88      0.81       533
Positive Review       0.86      0.72      0.78       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066



Overall the model is not bad he's correctly classifies 4 out of 5 reviews.

> 
This is pretty good for sentiment analysis 🤌

Here's a detailed analysis of this classification report:

**🔍 Class-by-Class Analysis**

Negative Review Performance

- **Precision: 0.76** (76%)

- When the model predicts "negative", it's correct 76% of the time

- **24% false positives** - sometimes it incorrectly labels positive reviews as negative

- **Recall: 0.88** (88%)

- The model finds 88% of all actual negative reviews

- **Only 12% false negatives** - rarely misses a negative review

- **F1-score: 0.81** - Good balance, but recall is stronger than precision

**Interpretation:** The model is **sensitive to negativity** - it catches almost all negative reviews but sometimes over-predicts negativity.

Positive Review Performance

- **Precision: 0.86** (86%)

- When the model predicts "positive", it's correct 86% of the time

- **14% false positives** - rarely mislabels negatives as positive

- **Recall: 0.72** (72%)

- The model only finds 72% of actual positive reviews

- **28% false negatives** - misses over 1 in 4 positive reviews!

- **F1-score: 0.78** - Slightly lower than negative class

**Interpretation:** The model is **conservative with positivity** - when it says positive, it's usually right, but it misses many positive reviews (probably labeling them as negative instead).

The Trade-off pattern we need to find

There's an **inverse relationship** between the classes:

- **Negative:** High recall (0.88), Lower precision (0.76) → *Over-predicts negative*

- **Positive:** High precision (0.86), Lower recall (0.72) → *Under-predicts positive*

This suggests the model has a **negative bias** - it's more likely to classify uncertain reviews as negative.

**📈 Aggregate Metrics**

- **Macro avg (0.81, 0.80, 0.80)**: Simple average across both classes

- **Weighted avg (0.81, 0.80, 0.80)**: Weighted by support (but since support is equal, same as macro)

- **Support: 533 each** - Perfectly balanced dataset, so no class imbalance issues

#### Conclusion
**Strengths:**

- ✅ 80% accuracy is solid

- ✅ Excellent at detecting negative sentiment (88% recall)

- ✅ When it predicts positive, it's usually right (86% precision)

**Weaknesses:**

- ⚠️ Misses 28% of positive reviews

- ⚠️ Has a slight negative bias

- ⚠️ Could improve positive review detection

👉 If false negatives on positive reviews are costly (e.g., missing 
happy customers), you might want to adjust the classification threshold 
or retrain the model to be less pessimistic!

## Classification tasks with embeddings
Now let's see how we can use embeddings to classify our data.

> 
What's aappening if we can not find a model that fits perfectly our needs ?

Then we need to fine-tune a model to our specific task, but it will be long hard and costly ... 😅

> 
So what's the solution ?

Use embeddings !

### Supervised classification with embeddings
Instead
 of using a pre-trained model for our specific task, we will use an 
embedding moidel for feature generation. Then those features will be 
used to train a classifier, this method is called **Supervised classification with embeddings** because we do not need to fine-tune the model, we just need to train a classifier on the features 🧙

For this example we will use a `sentence-transformers` model to generate embeddings for our data it's very popular and well-performing for this kind of task.

In [ ]:
from sentence_transformers import SentenceTransformer

# Sentence-level embedding model (frozen) used to generate features
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model = SentenceTransformer(embedding_model_name)

# Convert text to embeddings (NumPy arrays)
train_embeddings = model.encode(data["train"]["text"], show_progress_bar=True)
test_embeddings = model.encode(data["test"]["text"], show_progress_bar=True)

Batches: 100%|██████████| 34/34 [00:05<00:00,  6.70it/s]


In [ ]:
# Quick sanity check: encode a couple of sentences
sentences = ["This is an example sentence", "Each sentence is converted"]

embeddings = model.encode(sentences)
print(embeddings)

[[ 0.02250263 -0.07829173 -0.02303073 ... -0.00827931  0.02652684
  -0.00201897]
 [ 0.04170237  0.0010974  -0.01553417 ... -0.02181629 -0.06359361
  -0.00875287]]


In [9]:
train_embeddings.shape

(8530, 768)

This shape shows that each of our 8530 input documents has an embeddings dimension of 768 !

Now let's train a very simple logisitic regression on our embeddings 🤓

In [ ]:
from sklearn.linear_model import LogisticRegression

# Train a lightweight classifier on top of frozen embeddings
clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, data["train"]["label"])

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [11]:
# Predict previously unseen instances
y_pred = clf.predict(test_embeddings)
evaluate_performance(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.85      0.86      0.85       533
Positive Review       0.86      0.85      0.85       533

       accuracy                           0.85      1066
      macro avg       0.85      0.85      0.85      1066
   weighted avg       0.85      0.85      0.85      1066



Congrats with the embeddings training we acheive a better F1 score thant initially 🥳

> 
This demonstrate the possibility of training a light classifier while keeping the embeddings model frozen

### What if we do not have labeled data : unsupervised use case
What
 would happen if we would not use a classifier at all? Instead, we can 
average the embeddings per class and apply cosine similarity to predict 
which classes match the documents best 🧙

### Zero shot classification
A
 zero shot classification is when a model can classify text into 
categories it has never been explicitly trained on, simply by 
understanding the semantic relationship between the input text and 
candidate label descriptions.

In our case we do not have labeled data we will try to predict these 
labels of input text enven though the model was not trained on them 🥷

> 
To perform zero-shot classification with embeddings, there is a 
little trick that we can use. We can describe our labels based on what 
they should represent. For example, a negative label for movie reviews 
can be described as "This is a negative movie review." By describing and
 embedding the labels and documents, we have data that we can work with.

In [ ]:
# Create embeddings for our label descriptions
label_texts = ["A negative review", "A positive review"]
label_embeddings = model.encode(label_texts)

To assign labels to documents, we can apply cosine similarity to the document label pairs.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Assign the label whose embedding is closest (cosine similarity)
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

In [14]:
evaluate_performance(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.78      0.77      0.78       533
Positive Review       0.77      0.79      0.78       533

       accuracy                           0.78      1066
      macro avg       0.78      0.78      0.78      1066
   weighted avg       0.78      0.78      0.78      1066



AN F1 score at 0.78 is quite impressive considering we did not used 
any labels !! This is thez perfect illustration why embeddings can be a 
very usefull tool !

## Text classification with generative models
Generative
 language models like OpenAI's GPT differ fundamentally in their 
approach to classification compared to traditional methods.

Rather than following conventional classification paradigms, these models function as sequence-to-sequence systems in short : **they receive text input and produce text output.**

> 
While these generative models undergo training across diverse tasks, 
they typically cannot handle specialized use cases immediately. Consider
 feeding a movie review to such a model without additional guidance: the
 model would lack direction on how to process it.

To achieve meaningful results, we must provide context and steer the 
model toward our desired outcomes. This guidance occurs primarily 
through carefully crafted instructions, known as prompts 😎

For our demo we will use the [groq API](https://github.com/groq/groq-python) because openAI do not give us a free API keys 😅

In [ ]:
# Install the Groq Python client (needed only for the API demo below)
! pip install groq

In [ ]:
# IMPORTANT:
# - Put your API key in an environment variable named GROQ_API_KEY.
# - In practice, set it in your shell / notebook environment or in a `.env` file.
#
# This command is a placeholder example (it may not persist across notebook cells
# depending on your Jupyter environment):
! export GROQ_API_KEY="YOUR_API_KEY"

In [ ]:
# Take one example review to test prompts quickly
sample_text = data["test"]["text"][0]
print(f"Review: {sample_text}\n")

Review: lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .



In [ ]:
import os
from dotenv import load_dotenv
from groq import Groq

# Load environment variables from a local `.env` file if present
load_dotenv()

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Simple prompt: force the model to answer with a single label
chat_completion = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a sentiment classifier. Respond with only 'positive' or 'negative'.",
        },
        {
            "role": "user",
            "content": f"Classify the sentiment of this movie review: {sample_text}",
        },
    ],
    temperature=0,
    max_tokens=10,
)

print(chat_completion.choices[0].message.content)

positive.


or we can output a scroe if you need more granularity

In [ ]:
# Same idea, but return a numeric score in [0, 1] for finer granularity
model_name = "meta-llama/llama-4-scout-17b-16e-instruct"

messages = [
    {
        "role": "system",
        "content": (
            "You are a sentiment classifier. "
            "Rate the sentiment as a number between 0 (negative) and 1 (positive). "
            "Respond with only the number."
        ),
    },
    {
        "role": "user",
        "content": f"Rate the sentiment of this movie review: {sample_text}",
    },
]

chat_completion = client.chat.completions.create(
    model=model_name,
    messages=messages,
    temperature=0,
    max_tokens=10,
)
print(chat_completion.choices[0].message.content)

0.9


Let's evaluate this classifier with the same classification repport and see how it's performing 🤔

> 
Keep in mind this is a very simple prompt, if you need more control 
about the LLM output you can check how to structure the output of an LLM
 on the [openai doc](https://platform.openai.com/docs/guides/structured-outputs)

In [ ]:
def groq_generation(review_text: str, model: str = "meta-llama/llama-4-scout-17b-16e-instruct") -> str:
    """Return a sentiment score in [0, 1] from Groq (0=negative, 1=positive)."""
    messages = [
        {
            "role": "system",
            "content": (
                "You are a sentiment classifier. "
                "Rate the sentiment as a number between 0 (negative) and 1 (positive). "
                "Respond with only the number."
            ),
        },
        {
            "role": "user",
            "content": f"Rate the sentiment of this movie review: {review_text}",
        },
    ]

    chat_completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        max_tokens=10,
    )
    return chat_completion.choices[0].message.content

In [29]:
groq_generation(sample_text)

'0.8'

In [ ]:
# (Optional) Run the scorer on the full test set (slow + costs API credits):
# predictions = [groq_generation(doc) for doc in tqdm(data["test"]["text"])]

In [ ]:
#y_pred = [int(pred) for pred in predictions]
#evaluate_performance(data["test"]["label"], y_pred)

### Text2Text transfert transformers
Let's explore a final technique called [text-to-text transfert transformers](https://arxiv.org/abs/1910.10683) or T5 models. 👀 The architecture is similar to the original Transformers with ezncoder and decoder parts stacked together.

> 
T5 reframes every common NLP tasks such as translation, summarization, classification, question answering. **As input text → output text, simplifying model design and enabling multitask learning.**

T5 was trained on the [Colossal Clean Crawled Corpus](https://github.com/allenai/c4-documentation), with a self-supervised objective called [span corruption](https://arxiv.org/abs/2401.13160), giving it strong generalization across NLP tasks.

> 
Because T5 generates text tokens for answers and labels, it excels in
 zero-shot, few-shot, and instruction-based tasks, without needing 
task-specific heads or architectures 😎

In [ ]:
# Text-to-text classification with an instruction-tuned T5 model

# Reuse our device helper from above
device = select_torch_device()
print(f"Using device: {device}")

pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    device=device,
)

Using device: cpu


Device set to use cpu


In [ ]:
# Prepare our data by turning classification into a text-to-text prompt
prompt = "Is the following sentence positive or negative? "

def add_t5_prompt(example):
    return {"t5": prompt + example["text"]}

data = data.map(add_t5_prompt)
data

Map: 100%|██████████| 1066/1066 [00:00<00:00, 67463.27 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
})

Since this model generate text we need to map 0 for negative and 1 for positive then we can run our evaluation 🥸

In [ ]:
# Run inference
# The model generates text; we map "negative" -> 0 and anything else -> 1.
y_pred = []

test_prompts = KeyDataset(data["test"], "t5")
for output in tqdm(pipe(test_prompts), total=len(data["test"])):
    generated = output[0]["generated_text"]
    y_pred.append(0 if generated == "negative" else 1)

100%|██████████| 1066/1066 [00:52<00:00, 20.24it/s]


In [34]:
evaluate_performance(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.83      0.85      0.84       533
Positive Review       0.85      0.83      0.84       533

       accuracy                           0.84      1066
      macro avg       0.84      0.84      0.84      1066
   weighted avg       0.84      0.84      0.84      1066



GG for the Flan T5 model, 0.84 is a very good first F1 score 🥳

## Conclusion
Now
 I hope you have a better understanding of text classification and how 
to handle it with or without generative models. We know now that 
pretrained models are very good for classifying text !

We also know that we can leverage the power of embeddings to use it 
as input to train classifiers. Now in the next episode we'll explore the
 world of text clustering and topic modeling 🧙